In [7]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

base_url = "https://api.nhtsa.gov/SafetyRatings"
year = 2022

car_brands = ['Ford', 'Toyota', 'Tesla', 'Chevrolet', 'Honda', 'Nissan', 'Jeep', 'Hyundai', 'Subaru', 'GMC', 'Ram', 'Volkswagen', 'BMW', 'Mercedes-Benz', 'Dodge', 'Lexus', 'Audi', 'Mazda', 'Kia', 'Buick', 'Cadillac']


In [8]:
data = {}

for make in car_brands:
    # Fetch all models for the current make and year
    models_url = f"{base_url}/modelyear/{year}/make/{make}"
    response = requests.get(models_url)
    models_data = response.json()
    
    # Extract the list of models for the current make
    models = [model['Model'] for model in models_data['Results']]
    
    for model in models:
        # Fetch safety ratings for the current model
        safety_ratings_url = f"{base_url}/modelyear/{year}/make/{make}/model/{model}"
        response = requests.get(safety_ratings_url)
        safety_data = response.json()
        
        # Store the safety ratings for the current model
        data[f"{make} {model}"] = safety_data['Results']

In [9]:
# Flatten the nested dictionary structure
flattened_data = []
for make_model, attributes in data.items():
    for item in attributes:
        item['Make_Model'] = make_model
        flattened_data.append(item)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

In [10]:
print(df.columns)


Index(['VehicleDescription', 'VehicleId', 'Make_Model'], dtype='object')


In [11]:
alt_url = "https://api.nhtsa.gov/SafetyRatings/VehicleId"

# List to store overall ratings
overall_ratings = []

# Iterate through the 'VehicleId' column
for vehicle_id in df['VehicleId']:
    # Make the API call
    response = requests.get(f"{alt_url}/{vehicle_id}")
    data = response.json()
    
    # Extract the overall rating and append to the list
    overall_rating = data['Results'][0]['OverallRating']
    overall_ratings.append(overall_rating)

In [5]:
import requests
import pandas as pd

def fetch_nhtsa_recall_data_for_year(year):
    # Base URL for the NHTSA API for recall data
    rec_url = "https://api.nhtsa.gov/products/vehicle/recalls"

    # Define the parameters for the API request
    params = {
        'modelYear': year,
    }

    # Make the API request
    response = requests.get(rec_url, params=params)

    # Check for a successful response
    if response.status_code == 200:
        # Convert the JSON response to a DataFrame
        return pd.DataFrame(response.json()['results'])
    else:
        print(f"Failed to fetch data for year {year}: Status code {response.status_code}")
        return None



In [6]:
# Example usage: Fetch recall data for the year 2022
year = 2022
recall_data = fetch_nhtsa_recall_data_for_year(year)

# Check if data was fetched and display the first few rows
if recall_data is not None:
    print(recall_data.head())
else:
    print(f"No data fetched for year {year}.")


Failed to fetch data for year 2022: Status code 403
No data fetched for year 2022.


In [12]:
import requests
import pandas as pd

# Base URLs
base_url_safety_ratings = "https://api.nhtsa.gov/SafetyRatings"
base_url_recalls = "https://api.nhtsa.gov/recalls/recallsByVehicle"
year = 2022

car_brands = ['Ford', 'Toyota', 'Tesla', 'Chevrolet', 'Honda', 'Nissan', 'Jeep', 'Hyundai', 'Subaru', 'GMC', 'Ram', 'Volkswagen', 'BMW', 'Mercedes-Benz', 'Dodge', 'Lexus', 'Audi', 'Mazda', 'Kia', 'Buick', 'Cadillac']

# Data structures for storing results
safety_ratings_data = []
recall_data = []

for make in car_brands:
    # Fetch all models for the current make and year for safety ratings
    models_url = f"{base_url_safety_ratings}/modelyear/{year}/make/{make}"
    response = requests.get(models_url)
    models_data = response.json()

    # Extract the list of models for the current make
    models = [model['Model'] for model in models_data['Results']]

    for model in models:
        # Fetch safety ratings for the current model
        safety_ratings_url = f"{base_url_safety_ratings}/modelyear/{year}/make/{make}/model/{model}"
        response = requests.get(safety_ratings_url)
        safety_data = response.json()
        
        # Store the safety ratings for the current model
        for item in safety_data['Results']:
            item['Make'] = make
            item['Model'] = model
            safety_ratings_data.append(item)

        # Fetch recall data for the current make and model
        recall_response = requests.get(base_url_recalls, params={'make': make, 'model': model, 'modelYear': year})
        if recall_response.status_code == 200:
            recall_results = recall_response.json()['results']
            for recall in recall_results:
                recall['Make'] = make
                recall['Model'] = model
                recall_data.append(recall)

# Convert the data to DataFrames
df_safety_ratings = pd.DataFrame(safety_ratings_data)
df_recall_data = pd.DataFrame(recall_data)

# Displaying the first few rows of each DataFrame
print("Safety Ratings Data:")
print(df_safety_ratings.head())

print("\nRecall Data:")
print(df_recall_data.head())



Safety Ratings Data:
               VehicleDescription  VehicleId  Make         Model
0    2022 Ford Bronco 2DR SUV 4WD      17037  Ford        BRONCO
1    2022 Ford Bronco 4DR SUV 4WD      16331  Ford        BRONCO
2  2022 Ford Bronco Sport SUV 4WD      16332  Ford  BRONCO SPORT
3      2022 Ford EcoSport SUV AWD      16376  Ford      ECOSPORT
4      2022 Ford EcoSport SUV FWD      16333  Ford      ECOSPORT

Recall Data:
         Manufacturer NHTSACampaignNumber parkIt parkOutSide  \
0  Ford Motor Company           22V411000  False       False   
1  Ford Motor Company           22V451000  False       False   
2  Ford Motor Company           22V825000  False       False   
3  Ford Motor Company           22V928000  False       False   
4  Ford Motor Company           23V070000  False       False   

  ReportReceivedDate                                     Component  \
0         10/06/2022             LATCHES/LOCKS/LINKAGES:DOORS:LOCK   
1         24/06/2022                         VISIB

In [13]:
df_recall_data.to_csv("safety_recall_data.csv")